Modules import and set up.

In [ ]:
# import os, time, json, copy, pickle, random, requests, re
import numpy as np
# from PIL import Image

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from transformers import BertTokenizer, BertModel
from torch.utils.data import Dataset, DataLoader

In [ ]:
from tqdm import tqdm
import time
import pandas as pd

In [ ]:
import copy
from sklearn.model_selection import train_test_split

Read the movie and ratings files.

In [ ]:
# movie_code_list = list()
def create_films_dict(file_path):
    master_dict = {}
    with open(file_path, "r", encoding="latin-1") as file:
        for line in file:
            full_line = line.strip()
            id = int(full_line.split("::")[0])
            movie_code = full_line.split("::")[1]
            title = movie_code[0:-7]
            year = movie_code.split(" (")[-1][0:-1]
            char_to_find = ","
            if char_to_find in title:
                last_word = title.split(",")[-1].strip()
                if last_word == "The" or last_word == "A" or last_word == "An":
                    if len(title.split(",")) > 2:
                        true_title = f"{last_word} {title[0:-5]}"
                    else:
                        last_part = title.split(",")[0]
                        true_title = f"{last_word} {last_part}"
                    movie_code = f"{true_title} ({year})"
            imdb_filtered = imdb_data[imdb_data['movie_code'] == movie_code]
            info_stump = f"movieID:{id}; title:{movie_code}; year:{year}"
            if(imdb_filtered.shape[0] > 0):
                dur = imdb_filtered['duration'].iloc[0]
                coun = imdb_filtered['country'].iloc[0]
                lan = imdb_filtered['language'].iloc[0]
                dir = imdb_filtered['director'].iloc[0]
                wri = imdb_filtered['writer'].iloc[0]
                prod = imdb_filtered['production_company'].iloc[0]
                act = imdb_filtered['actors'].iloc[0]
                desc = imdb_filtered['description'].iloc[0]
                info_stump = f"{info_stump}; duration:{dur}; country{coun}; language:{lan}; director:{dir}; writers:{wri}; production_company:{prod}; actors:{act}; description:{desc}"
            master_dict[id] = info_stump
    return master_dict

def create_ratings_matrix(file_path):
    full_array = []
    with open(file_path, "r", encoding = "latin-1") as file:
        for line in file:
            full_line = line.strip()
            full_array.append(full_line)
        as_np_array = np.array(full_array)
    return as_np_array

In [ ]:
def parse_ratings(ratings_array):
    user_ids = []
    item_ids = []
    ratings = []

    for entry in ratings_array:
        parts = entry.split("::")
        user_ids.append(int(parts[0]))
        item_ids.append(int(parts[1]))
        ratings.append(float(parts[2]))

    return np.array(user_ids), np.array(item_ids), np.array(ratings)

In [ ]:
###DEBUGGING: EXCLUDED FROM SCRIPT
movies_path = "movies.dat"
ratings_path = "ratings.dat"
supp_path = "imdb_supp.csv"

imdb_data = pd.read_csv(supp_path)
imdb_data['movie_code'] = imdb_data['full_name_fstring'] = imdb_data.apply(lambda row: f"{row['original_title']} ({row['year']})", axis=1)

filmsdict = create_films_dict(movies_path)
ratingsmat = create_ratings_matrix(ratings_path)
user_ids, item_ids, ratings = parse_ratings(ratingsmat)
item_profiles = filmsdict

print(f"Loaded {len(user_ids)} ratings")
# print(item_profiles[0:5])
print(f"Loaded {len(item_profiles)} item profiles")

/tmp/ipython-input-1998688671.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  imdb_data = pd.read_csv(supp_path)


Loaded 1000209 ratings


KeyError: slice(0, 5, None)

In [ ]:
print(filmsdict)
item_profiles = prepare_item_profiles(filmsdict)
print(item_profiles)

{1: "movieID:1; title:Toy Story (1995); year:1995; duration:81; countryUSA; language:English; director:John Lasseter; writers:John Lasseter, Pete Docter; production_company:Pixar Animation Studios; actors:Tom Hanks, Tim Allen, Don Rickles, Jim Varney, Wallace Shawn, John Ratzenberger, Annie Potts, John Morris, Erik von Detten, Laurie Metcalf, R. Lee Ermey, Sarah Freeman, Penn Jillette, Jack Angel, Spencer Aste; description:A cowboy doll is profoundly threatened and jealous when a new spaceman figure supplants him as top toy in a boy's room.", 2: 'movieID:2; title:Jumanji (1995); year:1995; duration:104; countryUSA; language:English, French; director:Joe Johnston; writers:Jonathan Hensleigh, Greg Taylor; production_company:TriStar Pictures; actors:Robin Williams, Jonathan Hyde, Kirsten Dunst, Bradley Pierce, Bonnie Hunt, Bebe Neuwirth, David Alan Grier, Patricia Clarkson, Adam Hann-Byrd, Laura Bell Bundy, James Handy, Gillian Barber, Brandon Obray, Cyrus Thiedeke, Gary Joseph Thorup; de

Train and test data split. (80:20 split according to paper.)

In [ ]:
indices = np.arange(len(ratings))
train_idx, test_idx = train_test_split(indices, test_size=0.2, random_state=42)

train_users = user_ids[train_idx]
train_items = item_ids[train_idx]
train_ratings = ratings[train_idx]

test_users = user_ids[test_idx]
test_items = item_ids[test_idx]
test_ratings = ratings[test_idx]

print(f"Training samples: {len(train_ratings)}")
print(f"Test samples: {len(test_ratings)}")

NameError: name 'ratings' is not defined

Set device as GPU if possible.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

Reference: https://huggingface.co/google-bert/bert-base-uncased

In [ ]:
###DEBUGGING: EXCLUDED FROM SCRIPT
# Load BERT model
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased",
                                  output_hidden_states=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Generate vector with BERT.

In [ ]:
def get_vector_batch(texts, bert_model, bert_tokenizer, batch_size=16, device='cuda'):
  bert_model.eval()
  bert_model.to(device)
  all = []

  for i in tqdm(range(0, len(texts), batch_size)):
      batch = texts[i:i+batch_size]
      encoded = bert_tokenizer(batch, return_tensors='pt',
                                      padding=True, truncation=True,
                                      max_length=512)
      encoded = {k: v.to(device) for k, v in encoded.items()}

      with torch.no_grad():
          output = bert_model(**encoded)
          hidden_states = output.hidden_states
          # Sum last 4 layers
          last4 = torch.stack(hidden_states[-4:]).sum(0)
          # Mean pooling
          embedding = last4.mean(dim=1)
          all.append(embedding)

  return torch.cat(all, dim=0)

In [ ]:
def save_bert_vectors(filepath, bert_vecs):
    torch.save(bert_vecs.cpu(), filepath)
    print(f"Saved BERT vectors to {filepath}")
def load_bert_vectors(filepath, device="cuda"):
    vecs = torch.load(filepath, map_location=device)
    print(f"Loaded BERT vectors from {filepath}")
    return vecs

In [ ]:
###DEBUGGING: EXCLUDED FROM SCRIPT
vec = get_vector_batch(["hello world bro i dont know what else to Say what's wrong", "wha tis going on"])
# print(vec)
print(vec.shape)

100%|██████████| 1/1 [00:00<00:00,  3.29it/s]

torch.Size([2, 768])


Implement Deep AutoEncoder.

In [ ]:
class DeepAutoEncoder(nn.Module):
  # could be modified to just take in a dims list
  def __init__(self, input_dim=768, hidden_dim=64):
    super().__init__()
    self.encoder = nn.Sequential(
        nn.Linear(input_dim, 512),
        nn.Sigmoid(),
        nn.Linear(512, 256),
        nn.Sigmoid(),
        nn.Linear(256, 128),
        nn.Sigmoid(),
        nn.Linear(128, hidden_dim)
      )
    self.decoder = nn.Sequential(
        nn.Linear(hidden_dim, 128),
        nn.Sigmoid(),
        nn.Linear(128, 256),
        nn.Sigmoid(),
        nn.Linear(256, 512),
        nn.Sigmoid(),
        nn.Linear(512, input_dim),
      )
  def forward(self, x):
    reduced = self.encoder(x)
    reconstructed = self.decoder(reduced)
    return reduced, reconstructed

Dataloader.

In [ ]:
###DEBUGGING: EXCLUDED FROM SCRIPT
def get_dataloader(trainset, valset = None, batch_size = 64, num_workers = 2):
    train_loader = DataLoader(trainset, shuffle=True, num_workers=num_workers, batch_size=batch_size)
    if valset:
      val_loader = DataLoader(valset, shuffle=False, num_workers=num_workers, batch_size=batch_size)
      return (train_loader, val_loader)
    return (train_loader, None)

In [ ]:
class ItemProfileDataset(Dataset):
    def __init__(self,input):
        self.vecs = input

    def __len__(self):
        return self.vecs.size(0)

    def __getitem__(self, idx):
        v = self.vecs[idx]
        return v, v

Training DAE.

In [ ]:
def train_DAE_epoch(model, train_loader, optimizer, criterion, device):
  model.train()
  total_loss = 0.0

  for x, y in tqdm(train_loader, desc="Training DAE"):
      x = x.to(device)
      y = y.to(device)

      _, out = model.forward(x)
      loss = criterion(out, y)
      total_loss += loss.item()

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

  return total_loss / len(train_loader)

def train_DAE(model, trainset, device, n_epochs=10, batch_size=64, lr=0.001):
    model.to(device)
    train_loader = DataLoader(trainset, shuffle=True, batch_size=batch_size)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=5e-4)
    criterion = nn.MSELoss()

    best_loss = float('inf')
    best_state = None
    for epoch in range(n_epochs):
        avg_loss = train_DAE_epoch(model, train_loader,
                                          optimizer, criterion, device)
        print(f"Epoch {epoch+1}/{n_epochs} - Loss: {avg_loss:.6f}")

        if avg_loss < best_loss:
            best_loss = avg_loss
            best_state = copy.deepcopy(model.state_dict())
    if best_state is not None:
        model.load_state_dict(best_state)

    return model


In [ ]:
def save_dae_model(dae_model, filepath='dae_model.pt'):
    torch.save(dae_model.state_dict(), filepath)
    print(f"DAE model saved to {filepath}")

def train_and_save_dae(item_profiles_dict, item_ids,
                       hidden_dim=20, batch_size=64,
                       n_epochs=20, device='cuda',
                       save_path='dae_model.pt'):
    print("=== Training DAE Model ===")
    # load BERT
    bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    bert_model = BertModel.from_pretrained("bert-base-uncased",
                                          output_hidden_states=True)
    # prepare the supplementary item texts to one list
    unique_items = np.unique(item_ids)
    item_texts = []
    for item_id in unique_items:
        if item_id in item_profiles_dict:
            item_texts.append(item_profiles_dict[item_id])
        else:
            item_texts.append("")
    # generate BERT embeddings
    # print("generating BERT embeddings...")
    start = time.time()
    bert_vecs = get_vector_batch(item_texts, bert_model, bert_tokenizer,
                                 batch_size=batch_size, device=device)
    print(f"BERT time: {time.time() - start:.1f}s")
    # train DAE
    # print("Training Deep Autoencoder...")
    dae_model = DeepAutoEncoder(input_dim=768, hidden_dim=hidden_dim)
    dae_dataset = ItemProfileDataset(bert_vecs)
    start = time.time()
    dae_model = train_DAE(dae_model, dae_dataset,
                                  n_epochs=n_epochs,
                                  batch_size=batch_size,
                                  device=device)
    print(f"DAE time: {time.time() - start:.1f}s")

    # save model
    save_dae_model(dae_model, save_path)
    return dae_model

In [ ]:
###DEBUGGING: EXCLUDED FROM SCRIPT
def train_and_save_model(model, trainset, optimizer, device, batch_size=64, n_epochs=10, filepath=None):
    model.to(device)
    # train_loader, val_loader = get_dataloader(trainset, None, batch_size)
    if filepath is None:
        filepath = f"models/trained_{model.__class__.__name__}.pt"
    model = train_DAE(model, trainset, optimizer, device, n_epochs, batch_size)
    torch.save(model.state_dict(), filepath)
    return model

Using DAE to reduce dimensions.

In [ ]:
###DEBUGGING: EXCLUDED FROM SCRIPT
def get_reduced(model, input):
  data = torch.FloatTensor(input).to(device)
  model.eval()
  with torch.no_grad():
    reduced, _ = model.forward(data)
  return reduced

Implement factorization.

In [ ]:
def cosine_similarity(X):
    X_norm = X / (X.norm(dim=1, keepdim=True) + 1e-8)
    sim_matrix = X_norm @ X_norm.T
    return sim_matrix

In [ ]:
class MF(nn.Module):
    def __init__(self, num_users, num_items, num_factors=20, theta: torch.Tensor = None,  # [num_items, d]
                  k = 3,
                  beta= 0.01):
        """
        theta: the iteem profiles acquired from BERT+DAE
        """
        super().__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.f = num_factors
        self.beta = beta
        self.P = nn.Embedding(num_users, num_factors)
        self.Q = nn.Embedding(num_items, num_factors)

        #initialize randomly
        nn.init.normal_(self.P.weight, std=0.01)
        nn.init.normal_(self.Q.weight, std=0.01)

        if theta is not None:
            self.get_k_neighbors(theta, k)
        else:
            self.i_plus = None
            self.i_sims = None

        self.register_buffer("gamma", torch.ones(num_items))

    # get the k most similar items to each item
    def get_k_neighbors(self, theta, k):
        sim = cosine_similarity(theta)
        sim.fill_diagonal_(0)
        val, idx = torch.topk(sim, k=k, dim=1)
        self.register_buffer("i_plus", idx)
        self.register_buffer("i_sims", val)
        device = next(self.parameters()).device
        self.i_plus = self.i_plus.to(device)
        self.i_sims = self.i_sims.to(device)
        if torch.isinf(self.i_sims).sum().item() > 0 or torch.isnan(self.i_sims).sum().item() > 0:
            print("i_sums has inf or nan")

    def set_gamma(self, item_count, z=5.0):
        count = item_count.float().clamp_min(1.0)
        self.gamma = z / count
        # Clip gamma to prevent extreme values for items with very few ratings
        # self.gamma = torch.clamp(self.gamma, max=50.0)
        device = next(self.parameters()).device
        self.gamma = self.gamma.to(device)
        if torch.isinf(self.gamma).sum().item() > 0 or torch.isnan(self.gamma).sum().item() > 0:
            print("gamma has inf or nan")

    def forward(self, user_id, item_id):
        p = self.P(user_id)
        q = self.Q(item_id)
        return (p * q).sum(1)

    # objective function
    def loss(self, user_id, item_id, rating):
        pred = self.forward(user_id, item_id)
        error = 0.5 * (rating - pred).square().sum()

        p = self.P(user_id)
        q = self.Q(item_id)
        reg = self.beta * (p.square().sum() + q.square().sum())

        # this is the supplementary information regularization
        if (self.i_plus is not None) and (self.i_sims is not None):
            unique = torch.unique(item_id)
            q_self = self.Q(unique)
            neighbors = self.i_plus[unique]
            # q_neighbor_flat = self.Q(neighbors.flatten())
            # q_neighbor = q_neighbor_flat.view(neighbors.shape[0], neighbors.shape[1], -1)  # [N, k, num_factors]
            q_neighbor = self.Q(neighbors)
            diff = (q_self.unsqueeze(1) - q_neighbor).square().sum(dim=2)
            sim = self.i_sims[unique]
            gamma = self.gamma[unique].unsqueeze(1)
            reg_supp =  (diff * sim * gamma).sum()
            # reg_supp = torch.clamp(reg_supp, max=1000.0)
            # print("error:", error.item(),
            #       "reg_l2:", reg.item(),
            #       "reg_supp:", float(reg_supp))
            reg += reg_supp
        return error + reg




In [ ]:
def train_MF_epoch(model, train_loader, optimizer):
  model.train()
  total_loss = 0.0
  for u, i, r in tqdm(train_loader):
      u = u.to(device)
      i = i.to(device)
      r = r.to(device)

      optimizer.zero_grad()
      loss = model.loss(u, i, r)

      # Check for NaN loss
      if torch.isnan(loss):
          print(f"Warning: NaN loss detected, skipping this batch")
          break

      loss.backward()

      # Clip gradients to prevent explosion
      # nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
      # nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)

      optimizer.step()

      total_loss += loss.item()

  return total_loss / len(train_loader)

def train_MF(model, n_epochs, train_loader, optimizer):
  for epoch in range(1, n_epochs + 1):
      avg_loss = train_MF_epoch(model, train_loader, optimizer)
      print(f"Epoch {epoch:02d} | loss = {avg_loss:.4f}")
  return model


Construct ALDRS class.

In [ ]:
class RatingsDataset(Dataset):
  def __init__(self, user_ids, item_ids, ratings):
    self.user_ids = torch.as_tensor(user_ids, dtype=torch.long)
    self.item_ids = torch.as_tensor(item_ids, dtype=torch.long)
    self.ratings = torch.as_tensor(ratings,  dtype=torch.float32)

  def __len__(self):
    return len(self.ratings)

  def __getitem__(self, idx):
    return (
      self.user_ids[idx],
      self.item_ids[idx],
      self.ratings[idx],
    )

In [ ]:
def run_adlrs_pipeline(
    user_ids,
    item_ids,
    ratings,
    item_profiles_dict,
    dae_file=None,
    bert_file=None,
    num_factors=20,
    k_neighbors=3,
    z_gamma=5.0,
    batch_size=64,
    dae_epochs=20,
    mf_epochs=40,
    lr=1e-3,
    device='cuda'
):
    unique_users = np.unique(user_ids)
    unique_items = np.unique(item_ids)
    user2idx = {u: i for i, u in enumerate(unique_users)}
    item2idx = {i: j for j, i in enumerate(unique_items)}
    u_idx = np.array([user2idx[u] for u in user_ids])
    i_idx = np.array([item2idx[i] for i in item_ids])
    num_users = len(unique_users)
    num_items = len(unique_items)
    print(f"users: {num_users}, items: {num_items}, ratings: {len(ratings)}")

    # BERT
    if bert_file is not None:
        print(f"\n=== Step 1: loading pre-trained BERT from {bert_file} ===")
        bert_vecs = load_bert_vectors(bert_file)
    else:
        print("\n=== Step 1: generating BERT embeddings ===")
        bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
        bert_model = BertModel.from_pretrained("bert-base-uncased",
                                              output_hidden_states=True)
        item_texts = []
        for item_id in unique_items:
            if item_id in item_profiles_dict:
                item_texts.append(item_profiles_dict[item_id])
            else:
                item_texts.append("")
        bert_vecs = get_vector_batch(item_texts, bert_model, bert_tokenizer,
                                    batch_size=batch_size, device=device)
        save_bert_vectors("bert_item_vectors.pt", bert_vecs)

    # DAE
    if dae_file is not None:
        print(f"\n=== Step 2: loading pre-trained DAE from {dae_file} ===")
        dae_model = DeepAutoEncoder(input_dim=768, hidden_dim=num_factors)
        dae_model.load_state_dict(torch.load(dae_file, map_location=device))
        dae_model.to(device)
        print("DAE model loaded")
    else:
        print("\n=== Step 2: training DAE ===")
        dae_model = train_and_save_dae(item_profiles_dict, item_ids,
                       hidden_dim=20, batch_size=64,
                       n_epochs=10, device=device,
                       save_path='dae_model.pt')

    # reduce
    print("\n=== Step 3: reducing dimensions ===")
    dae_model.eval()
    with torch.no_grad():
        bert_vecs = bert_vecs.to(device)
        reduced, _ = dae_model(bert_vecs)

    # MF
    print("\n=== Step 4: training MF ===")
    model = MF(
        num_users=num_users,
        num_items=num_items,
        num_factors=num_factors,
        theta=reduced,
        k=k_neighbors,
        beta=0.01,
    ).to(device)
    item_counts = torch.bincount(
        torch.as_tensor(i_idx, dtype=torch.long, device=device),
        minlength=num_items,
    )
    model.set_gamma(item_counts, z=z_gamma)

    train_dataset = RatingsDataset(u_idx, i_idx, ratings)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    start = time.time()
    trained_model = train_MF(model, mf_epochs, train_loader,
                            optimizer)
    print(f"MF time: {time.time() - start:.1f}s")

    return {
        'model': trained_model,
        'theta': reduced,
        'user2idx': user2idx,
        'item2idx': item2idx,
        'dae_model': dae_model
    }

def get_rating_pred(model, user_id, item_id, user2idx, item2idx, device):
    model.eval()
    u_idx = torch.tensor([user2idx[u] for u in user_id], dtype=torch.long).to(device)
    i_idx = torch.tensor([item2idx[i] for i in item_id], dtype=torch.long).to(device)

    with torch.no_grad():
      pred = model.forward(u_idx, i_idx)
    return pred

In [ ]:
def save_adlrs_model(result, filepath='adlrs_model.pt', config=None):
    save_dict = {
        'mf_model_state_dict': result['model'].state_dict(),
        'user2idx': result['user2idx'],
        'item2idx': result['item2idx'],
        'theta': result['theta'],
        'num_users': result['model'].num_users,
        'num_items': result['model'].num_items,
        'num_factors': result['model'].f,
    }

    if config is not None:
        save_dict['config'] = config

    torch.save(save_dict, filepath)
    print(f"ADLRS model saved to {filepath}")

def load_adlrs_model(filepath, device='cuda'):
    checkpoint = torch.load(filepath, map_location=device)

    model = MF(
        num_users=checkpoint['num_users'],
        num_items=checkpoint['num_items'],
        num_factors=checkpoint['num_factors'],
        theta=checkpoint['theta'],
        k=checkpoint.get('config', {}).get('k_neighbors', 3),
        beta=checkpoint.get('config', {}).get('beta', 0.01)
    )

    model.load_state_dict(checkpoint['mf_model_state_dict'])
    model.to(device)
    model.eval()

    print(f"ADLRS model loaded from {filepath}")

    return {
        'model': model,
        'user2idx': checkpoint['user2idx'],
        'item2idx': checkpoint['item2idx'],
        'theta': checkpoint['theta'],
        'config': checkpoint.get('config', {})
    }

Implement evaluation metrics.

In [ ]:
def _to_numpy(x):
    if isinstance(x, torch.Tensor):
        return x.detach().cpu().numpy()
    return x
def MAE(y_true, y_pred):
    y_true = _to_numpy(y_true)
    y_pred = _to_numpy(y_pred)
    return np.mean(np.abs(y_pred - y_true))

def RMSE(y_true, y_pred):
    y_true = _to_numpy(y_true)
    y_pred = _to_numpy(y_pred)
    return np.sqrt(np.mean((y_pred - y_true) ** 2))

def HR(scores, user_items, N):
    hits = 0
    users = 0
    for user, relevant_items in user_items.items():
        if len(relevant_items) == 0:
            continue

        users += 1
        ranked= sorted(scores[user].items(),
                              key=lambda x: x[1],
                              reverse=True)

        topN_items = [item for item, _ in ranked[:N]]
        if any(item in topN_items for item in relevant_items):
            hits += 1
    if users == 0:
        return 0.0
    return hits / users


Full runthrough.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# LOAD DATA
movies_path = "movies.dat"
ratings_path = "ratings.dat"
supp_path = "imdb_supp.csv"

imdb_data = pd.read_csv(supp_path)
imdb_data['movie_code'] = imdb_data['full_name_fstring'] = imdb_data.apply(lambda row: f"{row['original_title']} ({row['year']})", axis=1)

filmsdict = create_films_dict(movies_path)
ratingsmat = create_ratings_matrix(ratings_path)
user_ids, item_ids, ratings = parse_ratings(ratingsmat)
item_profiles = filmsdict

print(f"Loaded {len(user_ids)} ratings")
# print(item_profiles[0:5])
print(f"Loaded {len(item_profiles)} item profiles")

#TRAIN-TEST SPLIT
indices = np.arange(len(ratings))
train_idx, test_idx = train_test_split(indices, test_size=0.2, random_state=42)

train_users = user_ids[train_idx]
train_items = item_ids[train_idx]
train_ratings = ratings[train_idx]

test_users = user_ids[test_idx]
test_items = item_ids[test_idx]
test_ratings = ratings[test_idx]

print(f"Training samples: {len(train_ratings)}")
print(f"Test samples: {len(test_ratings)}")


Using device: cuda


/tmp/ipython-input-150178990.py:14: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  imdb_data = pd.read_csv(supp_path)


Loaded 1000209 ratings
Loaded 3883 item profiles
Training samples: 800167
Test samples: 200042


In [ ]:
#DAE
USE_PRETRAINED_DAE = True  # set to True to use pre-trained DAE
DAE_PATH = "trained_dae.pt"

if not USE_PRETRAINED_DAE:
    print("\n=== train and save DAE Model ===")
    dae_model = train_and_save_dae(
        item_profiles_dict=item_profiles,
        item_ids=item_ids,
        hidden_dim=20,
        batch_size=64,
        n_epochs=20,
        device=device,
        save_path=DAE_PATH
    )
    print(f"DAE model saved to {DAE_PATH}")
    USE_PRETRAINED_DAE = True

In [ ]:
BERT_PATH = "bert_item_vectors.pt"
USE_BERT = True #change this once have the .pt file

In [ ]:
print("\n=== running ADLRS pipeline ===")
result = run_adlrs_pipeline(
    user_ids=train_users,
    item_ids=train_items,
    ratings=train_ratings,
    item_profiles_dict=item_profiles,
    dae_file=DAE_PATH if USE_PRETRAINED_DAE else None,  # load pre-trained or train new
    bert_file=BERT_PATH if USE_BERT else None,  # load pre-generated or no
    num_factors=20,
    k_neighbors=3,
    z_gamma=5.0,
    batch_size=64,
    dae_epochs=20,  # only used if dae_file is None
    mf_epochs=40,
    lr=1e-3,
    device=device
)

model = result['model']
user2idx = result['user2idx']
item2idx = result['item2idx']

#SAVE ADLRS
print("\n=== Saving ADLRS Model ===")
save_adlrs_model(
    result=result,
    filepath='adlrs_complete_model.pt',
    config={
        'num_factors': 20,
        'k_neighbors': 3,
        'z_gamma': 5.0,
        'beta': 0.01,
        'mf_epochs': 40,
        'lr': 1e-3
    }
)
print("Model saved successfully!")


=== Running ADLRS Pipeline ===
Users: 6040, Items: 3683, Ratings: 800167

=== Step 1: Loading Pre-trained BERT from bert_item_vectors.pt ===
Loaded BERT vectors from bert_item_vectors.pt with shape torch.Size([3683, 768]) on cuda

=== Step 2: Loading Pre-trained DAE from trained_dae.pt ===
DAE model loaded successfully

=== Step 3: Extracting Item Features ===

=== Step 4: Training Matrix Factorization ===


100%|██████████| 12503/12503 [00:36<00:00, 345.22it/s]


Epoch 01 | loss = 274.8907


100%|██████████| 12503/12503 [00:35<00:00, 347.68it/s]


Epoch 02 | loss = 40.3677


...

Epoch 38 | loss = 27.6982


100%|██████████| 12503/12503 [00:35<00:00, 352.90it/s]


Epoch 39 | loss = 27.6537


100%|██████████| 12503/12503 [00:35<00:00, 351.28it/s]


Epoch 40 | loss = 27.6126
MF time: 1420.8s

=== Saving ADLRS Model ===
ADLRS model saved to adlrs_complete_model.pt
Model saved successfully!


In [ ]:
# %%capture output_log
#TEST EVALUATION
print("\n=== Evaluating on Test Set ===")
test_mask = np.array([
    (u in user2idx and i in item2idx)
    for u, i in zip(test_users, test_items)
])

filtered_test_users = test_users[test_mask]
filtered_test_items = test_items[test_mask]
filtered_test_ratings = test_ratings[test_mask]
print(f"Test samples after filtering: {len(filtered_test_ratings)}")

test_predictions = get_rating_pred(
    model,
    filtered_test_users,
    filtered_test_items,
    user2idx,
    item2idx,
    device
)

if isinstance(test_predictions, torch.Tensor):
    test_predictions = test_predictions.detach().cpu().numpy()
test_mae = MAE(filtered_test_ratings, test_predictions)
test_rmse = RMSE(filtered_test_ratings, test_predictions)
print(f"\nTest MAE: {test_mae:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")

#ALL DATASET
print("\n=== Evaluating on All Dataset ===")
full_mask = np.array([
    (u in user2idx and i in item2idx)
    for u, i in zip(user_ids, item_ids)
])

filtered_users = user_ids[full_mask]
filtered_items = item_ids[full_mask]
filtered_ratings = ratings[full_mask]
print(f"Full data samples after filtering: {len(filtered_ratings)}")

predictions = get_rating_pred(
    model,
    filtered_users,
    filtered_items,
    user2idx,
    item2idx,
    device
)

if isinstance(predictions, torch.Tensor):
    predictions = predictions.detach().cpu().numpy()
mae = MAE(filtered_ratings, predictions)
rmse = RMSE(filtered_ratings, predictions)
print(f"\nFull dataset MAE: {mae:.4f}")
print(f"Full dataset RMSE: {rmse:.4f}")

# Cold Items FROM FULL DATASET (items with < 5 ratings)
print("\n=== Evaluating Cold Items ===")
from collections import Counter
item_counts = Counter(item_ids)
cold_item_mask = np.array([
    item_counts[i] < 5
    for i in filtered_items
])

if cold_item_mask.sum() > 0:
    cold_predictions = predictions[cold_item_mask]
    cold_ratings = filtered_ratings[cold_item_mask]
    cold_mae = MAE(cold_ratings, cold_predictions)
    cold_rmse = RMSE(cold_ratings, cold_predictions)
    print(f"Cold items count: {cold_item_mask.sum()}")
    print(f"Cold items MAE: {cold_mae:.4f}")
    print(f"Cold items RMSE: {cold_rmse:.4f}")
else:
    print("No cold items in dataset")

# Heavy Items FROM FULL DATASET (items with > 100 ratings)
print("\n=== Evaluating Heavy Items ===")
heavy_item_mask = np.array([
    item_counts[i] > 100
    for i in filtered_items
])

if heavy_item_mask.sum() > 0:
    heavy_predictions = predictions[heavy_item_mask]
    heavy_ratings = filtered_ratings[heavy_item_mask]
    heavy_mae = MAE(heavy_ratings, heavy_predictions)
    heavy_rmse = RMSE(heavy_ratings, heavy_predictions)
    print(f"Heavy items count: {heavy_item_mask.sum()}")
    print(f"Heavy items MAE: {heavy_mae:.4f}")
    print(f"Heavy items RMSE: {heavy_rmse:.4f}")
else:
    print("No heavy items in dataset")

# Controversial Items FROM FULL DATASET (rating standard deviation greater than 1.5)
print("\n=== Evaluating Controversial Items ===")
from collections import defaultdict
item_ratings = defaultdict(list)
for iid, r in zip(item_ids, ratings):
    item_ratings[iid].append(r)
item_std = {
    iid: np.array(rs, dtype=float).std()
    for iid, rs in item_ratings.items()
}
contr_item_mask = np.array([
    item_std[i] > 1.5
    for i in filtered_items
])

if contr_item_mask.sum() > 0:
    contr_predictions = predictions[contr_item_mask]
    contr_ratings = filtered_ratings[contr_item_mask]
    contr_mae = MAE(contr_ratings, contr_predictions)
    contr_rmse = RMSE(contr_ratings, contr_predictions)
    print(f"Controversial items count: {contr_item_mask.sum()}")
    print(f"Controversial items MAE: {contr_mae:.4f}")
    print(f"Controversial items RMSE: {contr_rmse:.4f}")
else:
    print("No controversial items in dataset")

# with open("adlrs_output.log", "w") as f:
#     f.write(output_log.stdout)


=== Evaluating on Test Set ===
Test samples after filtering: 200016

Test MAE: 0.6772
Test RMSE: 0.8594

=== Evaluating on All Dataset ===
Full data samples after filtering: 1000183

Full dataset MAE: 0.6200
Full dataset RMSE: 0.7861

=== Evaluating Cold Items ===
Cold items count: 572
Cold items MAE: 0.7527
Cold items RMSE: 0.9238

=== Evaluating Heavy Items ===
Heavy items count: 940925
Heavy items MAE: 0.6155
Heavy items RMSE: 0.7811

=== Evaluating Controversial Items ===
Controversial items count: 237
Controversial items MAE: 1.1185
Controversial items RMSE: 1.2688

=== Top-N Recommendation Example ===
Sample user ID: 5412

Top 10 recommendations:
1. The Matrix (1999) (score: 5.59)
2. Star Wars: Episode IV - A New Hope (1977) (score: 5.16)
3. The Wrong Trousers (1993) (score: 5.15)
4. A Close Shave (1995) (score: 5.12)
5. The Usual Suspects (1995) (score: 5.05)
6. The Princess Bride (1987) (score: 5.02)
7. Blade Runner (1982) (score: 4.97)
8. Seven Samurai (The Magnificent Seven)

In [ ]:
# HR@10
def evaluate_HR_at_10(model, test_users, test_items, test_ratings,
                     train_users, train_items,
                     user2idx, item2idx, all, device='cuda', batch_size=1000):
    print(f"\n=== Evaluating HR@10 ===")
    user_test_items = {}
    for user, item in zip(test_users, test_items):
        if user not in user_test_items:
            user_test_items[user] = set()
        user_test_items[user].add(item)

    user_train_items = {}
    for user, item in zip(train_users, train_items):
        if user not in user_train_items:
            user_train_items[user] = set()
        user_train_items[user].add(item)

    # print(f"number of test users: {len(user_test_items)}")

    valid = np.array([item for item in all if item in item2idx])
    # print(f"number of valid items for recommendation: {len(valid)}")
    scores = {}
    hits = 0
    users_evaluated = 0
    unique_test_users = list(user_test_items.keys())

    for user in tqdm(unique_test_users):
        if user not in user2idx:
            continue
        users_evaluated += 1
        exclude = user_train_items.get(user, set())
        candidate = [item for item in valid if item not in exclude]

        if len(candidate) == 0:
            continue
        user_array = np.full(len(candidate), user)
        item_array = np.array(candidate)
        predictions = get_rating_pred(
            model, user_array, item_array,
            user2idx, item2idx, device
        )

        if isinstance(predictions, torch.Tensor):
            predictions = predictions.detach().cpu().numpy()
        scores[user] = {item: float(score) for item, score in zip(candidate, predictions)}
        ground_truth = user_test_items[user]

        ranked = sorted(scores[user].items(), key=lambda x: x[1], reverse=True)
        top_10 = [item for item, _ in ranked[:10]]
        if any(item in top_10 for item in ground_truth):
            hits += 1
    if users_evaluated == 0:
        hr = 0.0
    else:
        hr = hits / users_evaluated

    print(f"\nHR@10 Results:")
    print(f"Users evaluated: {users_evaluated}")
    print(f"Hits: {hits}")
    print(f"HR@10: {hr:.4f}")
    return hr

hr_at_10 = evaluate_HR_at_10(
    model=model,
    test_users=test_users,
    test_items=test_items,
    test_ratings=test_ratings,
    train_users=train_users,
    train_items=train_items,
    user2idx=user2idx,
    item2idx=item2idx,
    all=np.array(list(item2idx.keys())),
    device=device,
    batch_size=1000
)

print(f"\nFinal HR@10: {hr_at_10:.4f}")


HR@10 Evaluation

=== Evaluating HR@10 ===
Number of test users: 6038
Number of valid items for recommendation: 3683


Evaluating users: 100%|██████████| 6038/6038 [00:28<00:00, 213.48it/s]


HR@10 Results:
  Users evaluated: 6038
  Hits: 3440
  HR@10: 0.5697

Final HR@10: 0.5697
